In [64]:
import numpy as np
import math
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader,TensorDataset

0 -> 0
90 ->3.14/2
180 -> 3.14

In [65]:
math.sin(3.14*1.5)

-0.999997146387718

In [66]:
3.14/180

0.017444444444444446

In [67]:
sin_data = [math.sin(x) for x in np.arange(0.0,5.0,0.01)]

In [68]:
len(sin_data)

500

In [90]:
input_dataset = [sin_data[x:x+8] for x in range(488)]
output_dataset = [sin_data[x:x+4] for x in range(8,496)]


In [91]:
print(sin_data[:13])

[0.0, 0.009999833334166664, 0.01999866669333308, 0.02999550020249566, 0.03998933418663416, 0.04997916927067833, 0.059964006479444595, 0.06994284733753277, 0.0799146939691727, 0.08987854919801104, 0.09983341664682815, 0.10977830083717481, 0.11971220728891936]


In [92]:
print(input_dataset[0])
print(output_dataset[0])

[0.0, 0.009999833334166664, 0.01999866669333308, 0.02999550020249566, 0.03998933418663416, 0.04997916927067833, 0.059964006479444595, 0.06994284733753277]
[0.0799146939691727, 0.08987854919801104, 0.09983341664682815, 0.10977830083717481]


In [93]:
print(len(input_dataset))
print(len(output_dataset))

488
488


In [94]:
train_input = input_dataset[:450]
train_output = output_dataset[:450]
test_input = input_dataset[450:]
test_output = output_dataset[450:]


In [95]:
test_output[-1]

[-0.9719030694018208,
 -0.9695006994538088,
 -0.967001380243766,
 -0.9644053617015305]

In [96]:
train_input_tensor = torch.FloatTensor(train_input).unsqueeze(-1)
train_output_tensor = torch.FloatTensor(train_output)
test_input_tensor = torch.FloatTensor(test_input).unsqueeze(-1)
test_output_tensor = torch.FloatTensor(test_output)


In [97]:
print("test_input_tensor shape:", test_input_tensor.shape)
print("test_output_tensor shape:", test_output_tensor.shape)

test_input_tensor shape: torch.Size([38, 8, 1])
test_output_tensor shape: torch.Size([38, 4])


In [98]:
test_input_tensor[-1]

tensor([[-0.9876],
        [-0.9860],
        [-0.9843],
        [-0.9825],
        [-0.9805],
        [-0.9785],
        [-0.9764],
        [-0.9742]])

In [99]:
train_tensor = TensorDataset(train_input_tensor,train_output_tensor)
test_tensor = TensorDataset(test_input_tensor,test_output_tensor)

In [100]:
class MYLSTM(nn.Module):
  def __init__(self,input_size=1, hidden_size = 50 , num_layers = 2, output_size = 4 ):

    super().__init__()

    self.lstm = nn.LSTM(input_size , hidden_size,num_layers, batch_first = True)

    self.fc = nn.Linear(hidden_size,output_size)

  def forward(self,x):
    lstm_output, (hidden,cell) = self.lstm(x)

    last_output = lstm_output[:,-1,:]

    output = self.fc(last_output)

    return output


In [101]:
train_tensor[0]


(tensor([[0.0000],
         [0.0100],
         [0.0200],
         [0.0300],
         [0.0400],
         [0.0500],
         [0.0600],
         [0.0699]]),
 tensor([0.0799, 0.0899, 0.0998, 0.1098]))

In [102]:
train_loader =  DataLoader(train_tensor,batch_size=32,shuffle = True)
test_loader =  DataLoader(test_tensor,batch_size=32,shuffle = True)

In [103]:
batch = next(iter(train_loader))
print(batch[0].shape)
print(batch[1].shape)

torch.Size([32, 8, 1])
torch.Size([32, 4])


In [104]:
def train(model,train_dataloader,test_dataloader, num_epochs = 100):
  device = torch.device('cuda' if torch.cuda_is_available() else 'cpu')
  model.to(device)

  criterion = nn.MSELoss()
  optimizer = optim.Adam(model.parameters(),lr = 0.001)

  train_losses = []
  test_losses = []

  for epoch in range(num_epochs):

    model.train()
    train_loss = 0

    for batch_idx,(inputs,target) in enumerate(train_loader):
      inputs = inputs.to(device)
      target = target.to(device)

      optimizer.zero_grad()
      output = model(inputs)
      loss = criterion(output,target)

      loss.backward()
      optimizer.step()

      train_loss += loss.item()
      train_losses.append(loss.item())

    model.eval()

    test_loss =0

    with torch.no_grad():
      for batch_idx,(inputs,target) in enumerate(test_loader):
        data,target = data.to(device) , target.to(device)
        output = model(data)

        test_loss += criterion(output,target).item()

        test_losses.append(test_loss.item())

    train_loss /= len(train_loader)
    test_loss /= len(test_loader)

    if (epoch+1)%10 == 0:
      print(f"Epoch[{(epoch+1)/num_epochs}] train loss is {train_loss:4f} test loss is {test_loss:4f}")



In [105]:
model = MYLSTM()

In [106]:
train_losses, val_losses = train_model(model, train_loader, test_loader, num_epochs=100)

Epoch 0: Train Loss: 0.464007, Val Loss: 1.139089
Epoch 20: Train Loss: 0.001677, Val Loss: 0.000094
Epoch 40: Train Loss: 0.001496, Val Loss: 0.000086
Epoch 60: Train Loss: 0.001483, Val Loss: 0.000078
Epoch 80: Train Loss: 0.001405, Val Loss: 0.000079
